Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [1]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/aina/.ivy2/cache
The jars for the packages stored in: /Users/aina/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0da620d0-2cbc-4586-8b2f-11c3b8be6bf8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 140ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [2]:
#Arxiu Parquet 
income = spark.read.parquet("./datalake/income_data/2024-04-17_IRSIncomeByZipCode_NoStateTotalsNoSmallZips.parquet")
income.show()


+-----+-------+-----------------+---------------------------+-----------+-----------------------------------+-------------------+----------------+-------------------------------------+---------------------+------------------+
|STATE|ZIPCODE|Number of returns|Adjusted gross income (AGI)|    Avg AGI|Number of returns with total income|Total income amount|Avg total income|Number of returns with taxable income|Taxable income amount|Avg taxable income|
+-----+-------+-----------------+---------------------------+-----------+-----------------------------------+-------------------+----------------+-------------------------------------+---------------------+------------------+
|   AL|  35004|             4930|                     255534|51.83245436|                               4930|             258024|     52.33752535|                                 4020|               163859|       40.76094527|
|   AL|  35005|             3300|                     128387|38.90515152|                       

In [3]:
from pyspark.sql.functions import col

# Lista de todos los caracteres inválidos que quieres reemplazar o eliminar
invalid_chars = [' ', ';', '{', '}', '(', ')', '\n', '\t', '=']

# Función para limpiar los nombres de las columnas reemplazando los caracteres no válidos
def clean_column_name(column_name):
    for invalid_char in invalid_chars:
        column_name = column_name.replace(invalid_char, "_")  # Reemplaza por subrayado o cualquier otro caracter válido que prefieras
    return column_name

# Aplicar la función de limpieza a cada columna
cleaned_sales = income.select([col(c).alias(clean_column_name(c)) for c in income.columns])

# Ahora guarda el DataFrame limpio en formato Delta
income.write.mode("overwrite").format("delta").save("./deltalake/income_data/")

AnalysisException: [DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES] Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema. Please use other characters and try again.